In [1]:
import math

from config import Config
from recommenders import HybridRecommender
from train_pipeline import (
    prepare_test_train_df,
    train_collaborative_filtering_model,
    train_content_based_master_game_model,
    train_content_based_feature_derived_model,
    evaluate_results
)

## Train basic recommenders

In [2]:
conf = Config.load_from_file()

# read users data
train_df, test_df = prepare_test_train_df(conf)

# train recommenders
collaborative_filtering_recommender, scaler = train_collaborative_filtering_model(train_df)
content_based_master_game_recommender = train_content_based_master_game_model(conf)
content_based_feature_derived_recommender = train_content_based_feature_derived_model(conf)
recommenders = [
    collaborative_filtering_recommender,
    content_based_master_game_recommender,
    content_based_feature_derived_recommender
]

## Search coefficients for the hybrid model

In [3]:
coef_range = [0, 0.01, 0.1, 0.2, 0.5, 0.8, 0.9, 0.99, 1.0]
results = []
for c1 in coef_range:
    for c2 in coef_range:
        for c3 in coef_range:
            if math.isclose(c1 + c2 + c3, 1, abs_tol=1e-6):
                coefficients = [c1, c2, c3]
                hybrid_recommender = HybridRecommender(recommenders, coefficients)
                metric = evaluate_results(
                    conf=conf,
                    recommender=hybrid_recommender,
                    scaler=scaler,
                    test_df=test_df

                )
                results.append((coefficients, metric))

## Top 10 results

In [4]:
results.sort(key=lambda x: x[1], reverse=True)
print(f"{' '*24}Coefficients{' '*24}| Metrics")
print(f"Collaborative filtering | Content-based 1 | Content-based 2 | Precision@{conf.precision_at_k}")
for (c1, c2, c3), metric in results[:10]:
    print(f"{c1:0.2f}{' '*20}| {c2:0.2f}{' '*12}| {c3:0.2f}{' '*12}| {metric: 0.3f}")

                        Coefficients                        | Metrics
Collaborative filtering | Content-based 1 | Content-based 2 | Precision@5
1.00                    | 0.00            | 0.00            |  0.438
0.99                    | 0.01            | 0.00            |  0.437
0.99                    | 0.00            | 0.01            |  0.437
0.90                    | 0.10            | 0.00            |  0.435
0.80                    | 0.20            | 0.00            |  0.426
0.90                    | 0.00            | 0.10            |  0.425
0.80                    | 0.10            | 0.10            |  0.422
0.80                    | 0.00            | 0.20            |  0.409
0.50                    | 0.50            | 0.00            |  0.397
0.50                    | 0.00            | 0.50            |  0.358
